In [1]:
from pathlib import Path
import pandas as pd


# Path to the data

In [4]:
BASE_DIR = Path.cwd()
train_data_dir = BASE_DIR / "msrp/msr_paraphrase_train.txt"


## Reading the file

In [46]:
def readfile(path):
    with path.open(mode="r", encoding="utf-8") as file:
        file_contents = file.read()

    return file_contents

# Preprocessing the data

In [47]:
def preprocess(path):
    data = readfile(path)
    d = data.split("\n")
    val = [i.split("\t") for i in d]
    return val


## Visualizating our data

In [55]:
data = preprocess(train_data_dir)

df = pd.DataFrame(data[1:], columns=data[0])
df = df.dropna()


In [59]:
df.head()

,﻿Quality,#1 ID,#2 ID,#1 String,#2 String
0,1,702876,702977,"Amrozi accused his brother, whom he called ""th...","Referring to him as only ""the witness"", Amrozi..."
1,0,2108705,2108831,Yucaipa owned Dominick's before selling the ch...,Yucaipa bought Dominick's in 1995 for $693 mil...
2,1,1330381,1330521,They had published an advertisement on the Int...,"On June 10, the ship's owners had published an..."
3,0,3344667,3344648,"Around 0335 GMT, Tab shares were up 19 cents, ...","Tab shares jumped 20 cents, or 4.6%, to set a ..."
4,1,1236820,1236712,"The stock rose $2.11, or about 11 percent, to ...",PG&E Corp. shares jumped $1.63 or 8 percent to...


In [56]:
df.count()

Quality     4076
#1 ID        4076
#2 ID        4076
#1 String    4076
#2 String    4076
dtype: int64

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4076 entries, 0 to 4075
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ﻿Quality   4076 non-null   object
 1   #1 ID      4076 non-null   object
 2   #2 ID      4076 non-null   object
 3   #1 String  4076 non-null   object
 4   #2 String  4076 non-null   object
dtypes: object(5)
memory usage: 191.1+ KB


In [58]:
x = df["#1 String"]
y = df["#2 String"]

